In [1]:
from __future__ import print_function
import os; os.environ['KERAS_BACKEND'] = 'tensorflow'
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation, Concatenate
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Input
from keras import backend as K
from keras.models import load_model
import pickle
import pandas as pd
import re
import numpy as np
import random
from keras.utils.vis_utils import plot_model
import keras.callbacks
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn import datasets, linear_model
import seaborn as sns
import pydotplus
#from numpy.random import seed; seed(123)
#import tensorflow as tf; tf.random.set_seed(123)
from sklearn.metrics import roc_auc_score
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import kruskal
from scipy.stats import ttest_ind
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_validate
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz, DecisionTreeRegressor


Using TensorFlow backend.
/home/huz/miniconda3/envs/CyTOF_DL/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [2]:
##### load data #####
processed_data = pickle.load( open( "result_04_processed_data_no_scale_Final.obj", "rb" ) )
cytof_files = processed_data["cytof_files"]
cytof_files.to_csv("result_05_cytof_files.csv")
expr_list = processed_data["expr_list"]
markerNames = processed_data["marker_names"]

r1 = [t1==t1 for t1 in cytof_files.CMV_Ab] 
cytof_files = cytof_files.loc[r1,:]
expr_list = expr_list[r1]
print(expr_list.shape)

y = cytof_files.CMV_Ab.values> 2
x = expr_list

(532, 10000, 27, 1)


In [3]:
##### split train, validation and test######

train_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i] not in ["SDY515","SDY519"]]
valid_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i]=="SDY515"]
test_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i]=="SDY519"]

x_train = x[train_id]
x_valid = x[valid_id]
x_test = x[test_id]

y_train = y[train_id]
y_valid = y[valid_id]
y_test = y[test_id]

In [4]:
##### define model #####
model_input = Input(shape=x_train[0].shape)
model_output = Conv2D(3, kernel_size=(1, x_train.shape[2]),
                 activation=None)(model_input)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)
#model_output = Dropout(0.3)(model_output)

model_output = Conv2D(3, (1, 1), activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)

model_output = AveragePooling2D(pool_size=(10000, 1))(model_output)
model_output = Flatten()(model_output)

model_output = Dense(3, activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("relu")(model_output)
model_output = Dense(1, activation=None)(model_output)
model_output = BatchNormalization()(model_output)
model_output = Activation("sigmoid")(model_output)


earlyStop = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=100, 
                                          verbose=0, mode='auto', baseline=None, restore_best_weights=True)

In [ ]:
##### 10 fold test######
valid_sdy = cytof_files.study_accession.unique()
result = pd.DataFrame(columns=['valid', 'test', 'auc'])

for i in range(10):
    sdy = np.random.choice(valid_sdy, size=2, replace=False)
    train_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i] not in sdy]
    valid_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i]==sdy[0]]
    test_id = [i for i in range(len(x)) if cytof_files.study_accession.iloc[i]==sdy[1]]

    x_train = x[train_id]
    x_valid = x[valid_id]
    x_test = x[test_id]

    y_train = y[train_id]
    y_valid = y[valid_id]
    y_test = y[test_id]
    
    checkpointer = keras.callbacks.ModelCheckpoint(filepath=('result_15/'+'weights'+str(i)+'.hdf5'), 
                                                   monitor='val_loss', verbose=0, save_best_only=True)
    
    model = keras.models.Model(inputs=[model_input],
                           outputs=model_output)
    
    model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

    model.fit([x_train], y_train,
          batch_size=60,
          epochs=10000,
          verbose=1,
          callbacks=[checkpointer,earlyStop],
          validation_data=([x_valid], y_valid))

    best_model = load_model('result_15/'+'weights'+str(i)+'.hdf5')

    y_true = y_test
    y_scores = best_model.predict([x_test])
    result = result.append({'valid':sdy[0] , 'test': sdy[1], 'auc': roc_auc_score(y_true, y_scores)}, 
                           ignore_index=True)
    
    result.to_csv("result_15_10fold_test.csv",index=False)
    
    # interpret
    dY = np.zeros([x_test.shape[0],x_test.shape[1]])
    s1 = np.random.randint(0,(x_test.shape[1]-1),int(x_test.shape[1]*0.05))

    for i2 in range(x_test.shape[0]):
        pred_i2 = best_model.predict([x_test[[i2],:,:,:]])
        for j in range(x_test.shape[1]):
            t1 = x_test[[i2],:,:,:].copy()
            t1[:,s1,:,:] = t1[:,j,:,:]
            pred_j = best_model.predict([t1])
            dY[i2,j] = pred_j-pred_i2

    # reformat dY
    x_test2 = x_test.reshape((x_test.shape[0]*x_test.shape[1],27))
    dY = dY.reshape([x_test.shape[0]*x_test.shape[1]])

    # Build decision tree to identify cell subset with high dY
    regr_1 = DecisionTreeRegressor(max_depth=4)
    regr_1.fit(x_test2, dY)

    # Plot the decision tree
    dot_data = StringIO()
    export_graphviz(regr_1, out_file=dot_data, 
                    feature_names= markerNames,
                    filled=True, rounded=True,
                    special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    graph.write_pdf(('result_15/'+'tree'+str(i)+'.pdf')) 

Train on 341 samples, validate on 71 samples
Epoch 1/10000
341/341 [==============================] - 3s 7ms/step - loss: 0.8242 - accuracy: 0.4751 - val_loss: 0.7725 - val_accuracy: 0.4648
Epoch 2/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.8223 - accuracy: 0.4751 - val_loss: 0.7607 - val_accuracy: 0.4648
Epoch 3/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.8205 - accuracy: 0.4428 - val_loss: 0.7496 - val_accuracy: 0.4648
Epoch 4/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.8205 - accuracy: 0.4575 - val_loss: 0.7389 - val_accuracy: 0.4648
Epoch 5/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.8345 - accuracy: 0.4633 - val_loss: 0.7290 - val_accuracy: 0.4648
Epoch 6/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.8292 - accuracy: 0.4457 - val_loss: 0.7198 - val_accuracy: 0.4648
Epoch 7/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.8094 -

Epoch 57/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7702 - accuracy: 0.5308 - val_loss: 0.8277 - val_accuracy: 0.4366
Epoch 58/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7759 - accuracy: 0.5308 - val_loss: 0.8349 - val_accuracy: 0.4366
Epoch 59/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7789 - accuracy: 0.4985 - val_loss: 0.8421 - val_accuracy: 0.4085
Epoch 60/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7848 - accuracy: 0.5220 - val_loss: 0.8494 - val_accuracy: 0.4225
Epoch 61/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7714 - accuracy: 0.5308 - val_loss: 0.8569 - val_accuracy: 0.4366
Epoch 62/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7840 - accuracy: 0.5220 - val_loss: 0.8642 - val_accuracy: 0.4366
Epoch 63/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7821 - accuracy: 0.5308 - val_loss: 0.8718 -

341/341 [==============================] - 1s 4ms/step - loss: 0.7477 - accuracy: 0.5308 - val_loss: 1.0856 - val_accuracy: 0.4930
Epoch 114/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7507 - accuracy: 0.5308 - val_loss: 1.0865 - val_accuracy: 0.4930
Epoch 115/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7488 - accuracy: 0.5396 - val_loss: 1.0873 - val_accuracy: 0.4930
Epoch 116/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7412 - accuracy: 0.5572 - val_loss: 1.0892 - val_accuracy: 0.4930
Epoch 117/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7540 - accuracy: 0.5543 - val_loss: 1.0897 - val_accuracy: 0.5070
Epoch 118/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7386 - accuracy: 0.5689 - val_loss: 1.0919 - val_accuracy: 0.5070
Epoch 119/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7468 - accuracy: 0.5484 - val_loss: 1.0933 - val_accu

Epoch 169/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7191 - accuracy: 0.5953 - val_loss: 1.1344 - val_accuracy: 0.4789
Epoch 170/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7282 - accuracy: 0.5630 - val_loss: 1.1342 - val_accuracy: 0.4789
Epoch 171/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7137 - accuracy: 0.5689 - val_loss: 1.1334 - val_accuracy: 0.4789
Epoch 172/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7145 - accuracy: 0.5894 - val_loss: 1.1358 - val_accuracy: 0.4789
Epoch 173/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7074 - accuracy: 0.5806 - val_loss: 1.1366 - val_accuracy: 0.4789
Epoch 174/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7134 - accuracy: 0.6070 - val_loss: 1.1398 - val_accuracy: 0.4789
Epoch 175/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.7189 - accuracy: 0.5630 - val_loss: 1

Epoch 225/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.6813 - accuracy: 0.6129 - val_loss: 1.1826 - val_accuracy: 0.4648
Epoch 226/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.6842 - accuracy: 0.6100 - val_loss: 1.1867 - val_accuracy: 0.4648
Epoch 227/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.6845 - accuracy: 0.5982 - val_loss: 1.1919 - val_accuracy: 0.4648
Epoch 228/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.6780 - accuracy: 0.6041 - val_loss: 1.1931 - val_accuracy: 0.4648
Epoch 229/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.6732 - accuracy: 0.6158 - val_loss: 1.1957 - val_accuracy: 0.4648
Epoch 230/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.6810 - accuracy: 0.6100 - val_loss: 1.1994 - val_accuracy: 0.4789
Epoch 231/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.6831 - accuracy: 0.5953 - val_loss: 1

...

341/341 [==============================] - 1s 4ms/step - loss: 0.3342 - accuracy: 0.8768 - val_loss: 1.6011 - val_accuracy: 0.5634
Epoch 1724/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3324 - accuracy: 0.8798 - val_loss: 1.6000 - val_accuracy: 0.5634
Epoch 1725/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3335 - accuracy: 0.8886 - val_loss: 1.6045 - val_accuracy: 0.5634
Epoch 1726/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3300 - accuracy: 0.8886 - val_loss: 1.6211 - val_accuracy: 0.5634
Epoch 1727/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3383 - accuracy: 0.8856 - val_loss: 1.6653 - val_accuracy: 0.5352
Epoch 1728/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3254 - accuracy: 0.8974 - val_loss: 1.6811 - val_accuracy: 0.5352
Epoch 1729/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3334 - accuracy: 0.8798 - val_loss: 1.6761 - va

341/341 [==============================] - 1s 4ms/step - loss: 0.3279 - accuracy: 0.8944 - val_loss: 1.7462 - val_accuracy: 0.5352
Epoch 1834/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3341 - accuracy: 0.8827 - val_loss: 1.7753 - val_accuracy: 0.5211
Epoch 1835/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3179 - accuracy: 0.8798 - val_loss: 1.7889 - val_accuracy: 0.5211
Epoch 1836/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3229 - accuracy: 0.8944 - val_loss: 1.7726 - val_accuracy: 0.5352
Epoch 1837/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3356 - accuracy: 0.8768 - val_loss: 1.7888 - val_accuracy: 0.5211
Epoch 1838/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3446 - accuracy: 0.8710 - val_loss: 1.7987 - val_accuracy: 0.5211
Epoch 1839/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3249 - accuracy: 0.8827 - val_loss: 1.8036 - va

341/341 [==============================] - 1s 4ms/step - loss: 0.3187 - accuracy: 0.8944 - val_loss: 1.8918 - val_accuracy: 0.5352
Epoch 1944/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3300 - accuracy: 0.8886 - val_loss: 1.8884 - val_accuracy: 0.5352
Epoch 1945/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3167 - accuracy: 0.8974 - val_loss: 1.8822 - val_accuracy: 0.5352
Epoch 1946/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3329 - accuracy: 0.8827 - val_loss: 1.8686 - val_accuracy: 0.5352
Epoch 1947/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3127 - accuracy: 0.9032 - val_loss: 1.8473 - val_accuracy: 0.5352
Epoch 1948/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3201 - accuracy: 0.8768 - val_loss: 1.8313 - val_accuracy: 0.5352
Epoch 1949/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3187 - accuracy: 0.8915 - val_loss: 1.8677 - va

341/341 [==============================] - 1s 4ms/step - loss: 0.3097 - accuracy: 0.8974 - val_loss: 1.9197 - val_accuracy: 0.5352
Epoch 2054/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3247 - accuracy: 0.8768 - val_loss: 1.9233 - val_accuracy: 0.5352
Epoch 2055/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3356 - accuracy: 0.8739 - val_loss: 1.9450 - val_accuracy: 0.5352
Epoch 2056/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3184 - accuracy: 0.8856 - val_loss: 1.9675 - val_accuracy: 0.5352
Epoch 2057/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3048 - accuracy: 0.8915 - val_loss: 1.9877 - val_accuracy: 0.5211
Epoch 2058/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3065 - accuracy: 0.8944 - val_loss: 1.9989 - val_accuracy: 0.5211
Epoch 2059/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3185 - accuracy: 0.8798 - val_loss: 1.9978 - va

341/341 [==============================] - 1s 4ms/step - loss: 0.3191 - accuracy: 0.8798 - val_loss: 2.1680 - val_accuracy: 0.4930
Epoch 2164/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3066 - accuracy: 0.8915 - val_loss: 2.1668 - val_accuracy: 0.4930
Epoch 2165/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3129 - accuracy: 0.8886 - val_loss: 2.1642 - val_accuracy: 0.4930
Epoch 2166/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3053 - accuracy: 0.8886 - val_loss: 2.1393 - val_accuracy: 0.4930
Epoch 2167/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3126 - accuracy: 0.8915 - val_loss: 2.1113 - val_accuracy: 0.5211
Epoch 2168/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3073 - accuracy: 0.8915 - val_loss: 2.0626 - val_accuracy: 0.5352
Epoch 2169/10000
341/341 [==============================] - 1s 4ms/step - loss: 0.3076 - accuracy: 0.8974 - val_loss: 2.0601 - va